In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

In [ ]:
pip install pywavelets

In [ ]:
%reset

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime as dt
from matplotlib import pyplot as plt
from scipy import signal
from scipy.signal import butter, lfilter, freqz,filtfilt, welch,find_peaks
from scipy.fft import fft 
from scipy.special import entr
from scipy.stats import kurtosis,skew,entropy
from scipy.stats.stats import pearsonr 
from scipy.fftpack import fft
import math
import time
import seaborn as sns
import pickle
from statistics import stdev
from statistics import variance
import random
from detecta import detect_peaks
import pywt 

# Loading already windowed data

In [ ]:

pickel_in1 = open('window_pi1.pickle','rb')
window1 = pickle.load(pickel_in1)
print(window1.head())

#pickel_in2 = open('window_pi1.pickle','rb')
#window2 = pickle.load(pickel_in2)
#print(window2.head())


In [ ]:
#pickel_in2 = open('window_pi2.pickle','rb')
#window2 = pickle.load(pickel_in2)
#print(window2.head())

# Adding the magnitude signal

In [ ]:

window1['magnitude'] =  (window1['acc_x']**2 + window1['acc_y']**2 + window1['acc_z']**2)**(1/2)
#window2['magnitude'] = (window2['acc_x']**2 + window2['acc_y']**2 + window2['acc_z']**2)**(1/2)

# Applying LowPass and HighPass Filter

In [ ]:
## Applying LowPass Filter


def butter_lowpass(cutoff, samp_freq, order = 3):
    nyq_freq = 0.5 * samp_freq
    normal_cutoff = cutoff/nyq_freq
    b,a = butter(order, normal_cutoff, btype = 'lowpass', analog = False)
    return b, a

def butter_lowpass_filter(data, cutoff, samp_freq, order = 3):
    b, a = butter_lowpass(cutoff, samp_freq, order)
    y = filtfilt(b, a, data)
    return y


order = 3
cutoff = 1
samp_freq = 20
b, a = butter_lowpass(cutoff, samp_freq, order)

body_x1 = butter_lowpass_filter(window1.acc_x, cutoff, samp_freq, order=3)
body_y1 = butter_lowpass_filter(window1.acc_y, cutoff, samp_freq, order=3)
body_z1 = butter_lowpass_filter(window1.acc_z, cutoff, samp_freq, order=3)
#gra_mag1 = butter_lowpass_filter(window1.magnitude, cutoff, samp_freq, order=3)

#gra_x2 = butter_lowpass_filter(window2.acc_x, cutoff, samp_freq, order=3)
#gra_y2 = butter_lowpass_filter(window2.acc_y, cutoff, samp_freq, order=3)
#gra_z2 = butter_lowpass_filter(window2.acc_z, cutoff, samp_freq, order=3)
#gra_mag2 = butter_lowpass_filter(window2.magnitude, cutoff, samp_freq, order=3)

window1['body_x'] = body_x1
window1['body_y'] = body_y1
window1['body_z'] = body_z1
window1['body_mag'] = (window1['body_x']**2 + window1['body_y']**2 + window1['body_z']**2)**(1/2)
#window2['gra_x'] = gra_x2
#window2['gra_y'] = gra_y2
#window2['gra_z'] = gra_z2
#window2['gra_mag'] = (window2['gra_x']**2 + window2['gra_y']**2 + window2['gra_z']**2)**(1/2)

In [ ]:
## Applying HighPass Filter


def butter_highpass(cutoff, samp_freq, order = 3):
    nyq_freq = 0.5 * samp_freq
    normal_cutoff = cutoff/nyq_freq
    b,a = butter(order, normal_cutoff, btype = 'highpass', analog = False)
    return b, a

def butter_highpass_filter(data, cutoff, samp_freq, order = 3):
    b, a = butter_highpass(cutoff, samp_freq, order)
    y = filtfilt(b, a, data)
    return y

order = 3
cutoff = 1
samp_freq = 20
b, a = butter_highpass(cutoff, samp_freq, order)

gra_x1 = butter_highpass_filter(window1.acc_x, cutoff, samp_freq, order=3)
gra_y1 = butter_highpass_filter(window1.acc_y, cutoff, samp_freq, order=3)
gra_z1 = butter_highpass_filter(window1.acc_z, cutoff, samp_freq, order=3)
#body_mag1 = butter_highpass_filter(window1.magnitude, cutoff, samp_freq, order=3)

#body_x2 = butter_highpass_filter(window2.acc_x, cutoff, samp_freq, order=3)
#body_y2 = butter_highpass_filter(window2.acc_y, cutoff, samp_freq, order=3)
#body_z2 = butter_highpass_filter(window2.acc_z, cutoff, samp_freq, order=3)
#body_mag2 = butter_highpass_filter(window2.magnitude, cutoff, samp_freq, order=3)

window1['gra_x'] = gra_x1
window1['gra_y'] = gra_y1
window1['gra_z'] = gra_z1
window1['gra_mag'] = (window1['gra_x']**2 + window1['gra_y']**2 + window1['gra_z']**2)**(1/2)
#window2['body_x'] = body_x2
#window2['body_y'] = body_y2
#window2['body_z'] = body_z2
#window2['body_mag'] = (window2['body_x']**2 + window2['body_y']**2 + window2['body_z']**2)**(1/2)

# Transforming into frequency domain

In [ ]:

## Transforming into frequency domain

# fft


def get_fft_values(y_values, T, N, f_s):
    fft_values = fft(y_values)
    fft_values = 2.0/N * np.abs(fft_values[0:N//2])
    return fft_values


t_n = 5 
N = 100
T = t_n / N
f_s = 1/T

window1['body_fredo_fft_magnitude_x'] = window1.body_x.apply(lambda x : get_fft_values(x, T, N,f_s))
window1['body_fredo_fft_magnitude_y'] = window1.body_y.apply(lambda x : get_fft_values(x, T, N,f_s))
window1['body_fredo_fft_magnitude_z'] = window1.body_z.apply(lambda x : get_fft_values(x, T, N,f_s))


window1['body_fredo_fft_magnitude_mag'] = (window1['body_fredo_fft_magnitude_x']**2 + window1['body_fredo_fft_magnitude_y']**2 + window1['body_fredo_fft_magnitude_z']**2)**(1/2)


# Power spectral Density


def fourier_magnitude(y, f , perseg):
    
    q, r = welch(x=y, fs=f, nperseg=perseg)
    return r

def fourier_frequency(y, f , perseg):
    
    q, r = welch(x=y, fs=f, nperseg=perseg)
    return q




samp_freq = 20
window_size = 100
window1['body_fredo_magnitude_x'] = window1.body_x.apply(lambda x : fourier_magnitude(x, f=samp_freq, perseg=window_size))
window1['body_fredo_frequency_x'] = window1.body_x.apply(lambda x : fourier_frequency(x, f=samp_freq, perseg=window_size))
window1['body_fredo_magnitude_y'] = window1.body_y.apply(lambda x : fourier_magnitude(x, f=samp_freq, perseg=window_size))
window1['body_fredo_frequency_y'] = window1.body_y.apply(lambda x : fourier_frequency(x, f=samp_freq, perseg=window_size))
window1['body_fredo_magnitude_z'] = window1.body_z.apply(lambda x : fourier_magnitude(x, f=samp_freq, perseg=window_size))
window1['body_fredo_frequency_z'] = window1.body_z.apply(lambda x : fourier_frequency(x, f=samp_freq, perseg=window_size))

window1['body_fredo_magnitude_mag'] = (window1['body_fredo_magnitude_x']**2 + window1['body_fredo_magnitude_y']**2 + window1['body_fredo_magnitude_z']**2)**(1/2)
window1['body_fredo_frequency_mag'] = (window1['body_fredo_frequency_x']**2 + window1['body_fredo_frequency_y']**2 + window1['body_fredo_frequency_z']**2)**(1/2)  


# auto-correlation

def autocorr(x):
    result = np.correlate(x,x , mode='full')
    return result[len(result)//2:]
 
def get_autocorr_values(y_values, T_a, N_a, f_s_a):
    autocorr_values = autocorr(y_values)
    x_values = np.array([T_a * jj for jj in range(0, N_a)])
    return autocorr_values

def get_autocorr_freq(y_values, T_a, N_a, f_s_a):
    autocorr_values = autocorr(y_values)
    x_values = np.array([T_a * jj for jj in range(0, N_a)])
    return 1/x_values
 
t_a = 5
N_a = 100
T_a = t_a / N_a
f_s_a = 1/T_a

window1['body_fredo_autcor_magnitude_x'] = window1.body_x.apply(lambda x : get_autocorr_values(x, T_a, N_a,f_s_a))
window1['body_fredo_autcor_frequency_x'] = window1.body_x.apply(lambda x : get_autocorr_freq(x, T_a, N_a,f_s_a))
window1['body_fredo_autcor_magnitude_y'] = window1.body_y.apply(lambda x : get_autocorr_values(x, T_a, N_a,f_s_a))
window1['body_fredo_autcor_frequency_y'] = window1.body_y.apply(lambda x : get_autocorr_freq(x, T_a, N_a,f_s_a))
window1['body_fredo_autcor_magnitude_z'] = window1.body_z.apply(lambda x : get_autocorr_values(x, T_a, N_a,f_s_a))
window1['body_fredo_autcor_frequency_z'] = window1.body_z.apply(lambda x : get_autocorr_freq(x, T_a, N_a,f_s_a))

                    
    
window1['body_fredo_autcor_magnitude_mag'] = (window1['body_fredo_autcor_magnitude_x']**2 + window1['body_fredo_autcor_magnitude_y']**2 + window1['body_fredo_autcor_magnitude_z']**2)**(1/2)
window1['body_fredo_autcor_frequency_mag'] = (window1['body_fredo_autcor_frequency_x']**2 + window1['body_fredo_autcor_frequency_y']**2 + window1['body_fredo_autcor_frequency_z']**2)**(1/2) 



# wavelet transformation


def get_wave_appro(signall,wavi,leve):
    cA,cD7,cD6,cD5,cD4,cD3,cD2, cD1 = pywt.wavedec(signall,wavelet=wavi, level=leve)
    return cA
def get_wave_details(signall,wavi,leve):
    cA,cD7,cD6,cD5,cD4,cD3,cD2, cD1 = pywt.wavedec(signall,wavelet=wavi, level=leve)
    return cD7

window1['wave_appro_x'] = window1.acc_x.apply(lambda x : get_wave_appro(x,'sym2',7))
window1['wave_details_x'] = window1.acc_x.apply(lambda x : get_wave_details(x,'sym2',7))
window1['wave_appro_x_1'] = window1.acc_x.apply(lambda x : get_wave_appro(x,'sym2',7)[0])
window1['wave_details_x_1'] = window1.acc_x.apply(lambda x : get_wave_details(x,'sym2',7)[0])
window1['wave_appro_x_2'] = window1.acc_x.apply(lambda x : get_wave_appro(x,'sym2',7)[1])
window1['wave_details_x_2'] = window1.acc_x.apply(lambda x : get_wave_details(x,'sym2',7)[1])
window1['wave_appro_x_3'] = window1.acc_x.apply(lambda x : get_wave_appro(x,'sym2',7)[2])
window1['wave_details_x_3'] = window1.acc_x.apply(lambda x : get_wave_details(x,'sym2',7)[2])

window1['wave_appro_y'] = window1.acc_y.apply(lambda x : get_wave_appro(x,'sym2',7))
window1['wave_details_y'] = window1.acc_y.apply(lambda x : get_wave_details(x,'sym2',7))
window1['wave_appro_y_1'] = window1.acc_y.apply(lambda x : get_wave_appro(x,'sym2',7)[0])
window1['wave_details_y_1'] = window1.acc_y.apply(lambda x : get_wave_details(x,'sym2',7)[0])
window1['wave_appro_y_2'] = window1.acc_y.apply(lambda x : get_wave_appro(x,'sym2',7)[1])
window1['wave_details_y_2'] = window1.acc_y.apply(lambda x : get_wave_details(x,'sym2',7)[1])
window1['wave_appro_y_3'] = window1.acc_y.apply(lambda x : get_wave_appro(x,'sym2',7)[2])
window1['wave_details_y_3'] = window1.acc_y.apply(lambda x : get_wave_details(x,'sym2',7)[2])

window1['wave_appro_z'] = window1.acc_z.apply(lambda x : get_wave_appro(x,'sym2',7))
window1['wave_details_z'] = window1.acc_z.apply(lambda x : get_wave_details(x,'sym2',7))
window1['wave_appro_z_1'] = window1.acc_z.apply(lambda x : get_wave_appro(x,'sym2',7)[0])
window1['wave_details_z_1'] = window1.acc_z.apply(lambda x : get_wave_details(x,'sym2',7)[0])
window1['wave_appro_z_2'] = window1.acc_z.apply(lambda x : get_wave_appro(x,'sym2',7)[1])
window1['wave_details_z_2'] = window1.acc_z.apply(lambda x : get_wave_details(x,'sym2',7)[1])
window1['wave_appro_z_3'] = window1.acc_z.apply(lambda x : get_wave_appro(x,'sym2',7)[2])
window1['wave_details_z_3'] = window1.acc_z.apply(lambda x : get_wave_details(x,'sym2',7)[2])

window1['wave_appro_mag'] = window1.magnitude.apply(lambda x : get_wave_appro(x,'sym2',7))
window1['wave_details_mag'] = window1.magnitude.apply(lambda x : get_wave_details(x,'sym2',7))
window1['wave_appro_mag_1'] = window1.magnitude.apply(lambda x : get_wave_appro(x,'sym2',7)[0])
window1['wave_details_mag_1'] = window1.magnitude.apply(lambda x : get_wave_details(x,'sym2',7)[0])
window1['wave_appro_mag_2'] = window1.magnitude.apply(lambda x : get_wave_appro(x,'sym2',7)[1])
window1['wave_details_mag_2'] = window1.magnitude.apply(lambda x : get_wave_details(x,'sym2',7)[1])
window1['wave_appro_mag_3'] = window1.magnitude.apply(lambda x : get_wave_appro(x,'sym2',7)[2])
window1['wave_details_mag_3'] = window1.magnitude.apply(lambda x : get_wave_details(x,'sym2',7)[2])





# Determining jerk signals

In [ ]:


## Determining Jerk

window1['jerk_x']= window1.acc_x.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_y']= window1.acc_y.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_z']= window1.acc_z.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_magnitude']= window1.magnitude.apply(lambda x : (np.diff(x)/(1/samp_freq)))

window1['jerk_body_fredo_magnitude_x']= window1.body_fredo_magnitude_x.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_body_fredo_magnitude_y']= window1.body_fredo_magnitude_y.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_body_fredo_magnitude_z']= window1.body_fredo_magnitude_z.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_body_fredo_magnitude_mag']= window1.body_fredo_magnitude_mag.apply(lambda x : (np.diff(x)/(1/samp_freq)))

window1['jerk_body_fredo_frequency_x']= window1.body_fredo_frequency_x.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_body_fredo_frequency_y']= window1.body_fredo_frequency_y.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_body_fredo_frequency_z']= window1.body_fredo_frequency_z.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_body_fredo_frequency_mag']= window1.body_fredo_frequency_mag.apply(lambda x : (np.diff(x)/(1/samp_freq)))

window1['jerk_body_fredo_fft_magnitude_x']= window1.body_fredo_fft_magnitude_x.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_body_fredo_fft_magnitude_y']= window1.body_fredo_fft_magnitude_y.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_body_fredo_fft_magnitude_z']= window1.body_fredo_fft_magnitude_z.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_body_fredo_fft_magnitude_mag']= window1.body_fredo_fft_magnitude_mag.apply(lambda x : (np.diff(x)/(1/samp_freq)))


window1['jerk_body_fredo_autcor_magnitude_x']= window1.body_fredo_autcor_magnitude_x.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_body_fredo_autcor_magnitude_y']= window1.body_fredo_autcor_magnitude_y.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_body_fredo_autcor_magnitude_z']= window1.body_fredo_autcor_magnitude_z.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_body_fredo_autcor_magnitude_mag']= window1.body_fredo_autcor_magnitude_mag.apply(lambda x : (np.diff(x)/(1/samp_freq)))


window1['jerk_body_fredo_autcor_frequency_x']= window1.body_fredo_autcor_frequency_x.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_body_fredo_autcor_frequency_y']= window1.body_fredo_autcor_frequency_y.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_body_fredo_autcor_frequency_z']= window1.body_fredo_autcor_frequency_z.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_body_fredo_autcor_frequency_mag']= window1.body_fredo_autcor_frequency_mag.apply(lambda x : (np.diff(x)/(1/samp_freq)))



window1['jerk_body_x']= window1.body_x.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_body_y']= window1.body_y.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_body_z']= window1.body_z.apply(lambda x : (np.diff(x)/(1/samp_freq)))
window1['jerk_body_mag']= window1.body_mag.apply(lambda x : (np.diff(x)/(1/samp_freq)))





# Determining Frequency Domain Features

In [ ]:
## Determining Frequency Domain Features


## kurtosis


window1['body_fredo_mag_kurtosis_x'] = window1.body_fredo_magnitude_x.apply(lambda x: kurtosis(x, fisher = True))
window1['body_fredo_mag_kurtosis_y'] = window1.body_fredo_magnitude_y.apply(lambda x: kurtosis(x, fisher = True))
window1['body_fredo_mag_kurtosis_z'] = window1.body_fredo_magnitude_z.apply(lambda x: kurtosis(x, fisher = True))
window1['body_fredo_mag_kurtosis_mag'] = window1.body_fredo_magnitude_mag.apply(lambda x: kurtosis(x, fisher = True))

window1['jerk_body_fredo_mag_kurtosis_x'] = window1.jerk_body_fredo_magnitude_x.apply(lambda x: kurtosis(x, fisher = True))
window1['jerk_body_fredo_mag_kurtosis_y'] = window1.jerk_body_fredo_magnitude_y.apply(lambda x: kurtosis(x, fisher = True))
window1['jerk_body_fredo_mag_kurtosis_z'] = window1.jerk_body_fredo_magnitude_z.apply(lambda x: kurtosis(x, fisher = True))
window1['jerk_body_fredo_mag_kurtosis_mag'] = window1.jerk_body_fredo_magnitude_mag.apply(lambda x: kurtosis(x, fisher = True))




## Sknewness



window1['body_fredo_mag_skew_x'] = window1.body_fredo_magnitude_x.apply(lambda x: skew(x))
window1['body_fredo_mag_skew_y'] = window1.body_fredo_magnitude_y.apply(lambda x: skew(x))
window1['body_fredo_mag_skew_z'] = window1.body_fredo_magnitude_z.apply(lambda x: skew(x))
window1['body_fredo_mag_skew_mag'] = window1.body_fredo_magnitude_mag.apply(lambda x: skew(x))

window1['jerk_body_fredo_mag_skew_x'] = window1.jerk_body_fredo_magnitude_x.apply(lambda x: skew(x))
window1['jerk_body_fredo_mag_skew_y'] = window1.jerk_body_fredo_magnitude_y.apply(lambda x: skew(x))
window1['jerk_body_fredo_mag_skew_z'] = window1.jerk_body_fredo_magnitude_z.apply(lambda x: skew(x))
window1['jerk_body_fredo_mag_skew_mag'] = window1.jerk_body_fredo_magnitude_mag.apply(lambda x: skew(x))



## Entropy



def get_entropy(x, axis = 0):
    x = x / np.sum(x, axis = axis , keepdims = True)
    entropy = np.sum(entr(x), axis = axis)
    return entropy


window1['body_fredo_mag_entropy_x'] = window1.body_fredo_magnitude_x.apply(lambda x: get_entropy(x))
window1['body_fredo_mag_entropy_y'] = window1.body_fredo_magnitude_y.apply(lambda x: get_entropy(x))
window1['body_fredo_mag_entropy_z'] = window1.body_fredo_magnitude_z.apply(lambda x: get_entropy(x))
window1['body_fredo_mag_entropy_mag'] = window1.body_fredo_magnitude_mag.apply(lambda x: get_entropy(x))


#window1['jerk_body_fredo_mag_entropy_x'] = window1.jerk_body_fredo_magnitude_x.apply(lambda x: get_entropy(x))
#window1['jerk_body_fredo_mag_entropy_y'] = window1.jerk_body_fredo_magnitude_y.apply(lambda x: get_entropy(x))
#window1['jerk_body_fredo_mag_entropy_z'] = window1.jerk_body_fredo_magnitude_z.apply(lambda x: get_entropy(x))
#window1['jerk_body_fredo_mag_entropy_mag'] = window1.jerk_body_fredo_magnitude_mag.apply(lambda x: get_entropy(x))

#window1['wave_app_entropy_x'] = window1.wave_appro_x.apply(lambda x : get_entropy(x))
#window1['wave_app_entropy_y'] = window1.wave_appro_y.apply(lambda x : get_entropy(x))
#window1['wave_app_entropy_z'] = window1.wave_appro_z.apply(lambda x : get_entropy(x))
#window1['wave_app_entropy_mag'] = window1.wave_appro_mag.apply(lambda x : get_entropy(x))
#window1['wave_det_entropy_x'] = window1.wave_details_x.apply(lambda x : get_entropy(x))
#window1['wave_det_entropy_y'] = window1.wave_details_y.apply(lambda x : get_entropy(x))
#window1['wave_det_entropy_z'] = window1.wave_details_z.apply(lambda x : get_entropy(x))
#window1['wave_det_entropy_mag'] = window1.wave_details_mag.apply(lambda x : get_entropy(x))




# Finding peak frequencies


z1 = window1.body_fredo_magnitude_x.apply(lambda x : find_peaks(x))
z2 = window1.body_fredo_magnitude_y.apply(lambda x : find_peaks(x))
z3 = window1.body_fredo_magnitude_z.apply(lambda x : find_peaks(x))
z4 = window1.body_fredo_magnitude_mag.apply(lambda x : find_peaks(x))
z5 = window1.jerk_body_fredo_magnitude_x.apply(lambda x : find_peaks(x))
z6 = window1.jerk_body_fredo_magnitude_y.apply(lambda x : find_peaks(x))
z7 = window1.jerk_body_fredo_magnitude_z.apply(lambda x : find_peaks(x))
z8 = window1.jerk_body_fredo_magnitude_mag.apply(lambda x : find_peaks(x))

a1 = window1.body_fredo_autcor_magnitude_x.apply(lambda x : find_peaks(x))
a2 = window1.body_fredo_autcor_magnitude_y.apply(lambda x : find_peaks(x))
a3 = window1.body_fredo_autcor_magnitude_z.apply(lambda x : find_peaks(x))
a4 = window1.body_fredo_autcor_magnitude_mag.apply(lambda x : find_peaks(x))
a5 = window1.jerk_body_fredo_autcor_magnitude_x.apply(lambda x : find_peaks(x))
a6 = window1.jerk_body_fredo_autcor_magnitude_y.apply(lambda x : find_peaks(x))
a7 = window1.jerk_body_fredo_autcor_magnitude_z.apply(lambda x : find_peaks(x))
a8 = window1.jerk_body_fredo_autcor_magnitude_mag.apply(lambda x : find_peaks(x))


match1 = []
match2 = []
match3 = []
match4 = []
match5 = []
match6 = []
match7 = []
match8 = []

auch1 = []
auch2 = []
auch3 = []
auch4 = []
auch5 = []
auch6 = []
auch7 = []
auch8 = []

for i in tqdm(range(len(z1))):
    match1.append(np.take(window1.body_fredo_frequency_x[i],z1[i][0]))
    match2.append(np.take(window1.body_fredo_frequency_y[i],z2[i][0]))
    match3.append(np.take(window1.body_fredo_frequency_z[i],z3[i][0]))
    match4.append(np.take(window1.body_fredo_frequency_mag[i],z4[i][0]))
    match5.append(np.take(window1.jerk_body_fredo_frequency_x[i],z5[i][0]))
    match6.append(np.take(window1.jerk_body_fredo_frequency_y[i],z6[i][0]))
    match7.append(np.take(window1.jerk_body_fredo_frequency_z[i],z7[i][0]))
    match8.append(np.take(window1.jerk_body_fredo_frequency_mag[i],z8[i][0]))
    auch1.append(np.take(window1.body_fredo_autcor_frequency_x[i],z1[i][0]))
    auch2.append(np.take(window1.body_fredo_autcor_frequency_y[i],z1[i][0]))
    auch3.append(np.take(window1.body_fredo_autcor_frequency_z[i],z1[i][0]))
    auch4.append(np.take(window1.body_fredo_autcor_frequency_mag[i],z1[i][0]))
    auch5.append(np.take(window1.jerk_body_fredo_autcor_frequency_x[i],z1[i][0]))
    auch6.append(np.take(window1.jerk_body_fredo_autcor_frequency_y[i],z1[i][0]))
    auch7.append(np.take(window1.jerk_body_fredo_autcor_frequency_z[i],z1[i][0]))
    auch8.append(np.take(window1.jerk_body_fredo_autcor_frequency_mag[i],z1[i][0]))
    
    
window1['peak_frequencies_x'] = match1
window1['peak_frequencies_y'] = match2
window1['peak_frequencies_z'] = match3
window1['peak_frequencies_mag'] = match4

window1['jerk_peak_frequencies_x'] = match5
window1['jerk_peak_frequencies_y'] = match6
window1['jerk_peak_frequencies_z'] = match7
window1['jerk_peak_frequencies_mag'] = match8

window1['autcor_peak_frequencies_x'] = auch1
window1['autcor_peak_frequencies_y'] = auch2
window1['autcor_peak_frequencies_z'] = auch3
window1['autcor_peak_frequencies_mag'] = auch4

window1['jerk_autcor_peak_frequencies_x'] = auch5
window1['jerk_autcor_peak_frequencies_y'] = auch6
window1['jerk_autcor_peak_frequencies_z'] = auch7
window1['jerk_autcor_peak_frequencies_mag'] = auch8


# Find peak magnitudes 


match9 = []
match10 = []
match11 = []
match12 = []
match13 = []
match14 = []
match15 = []
match16 = []

auch9 = []
auch10 = []
auch11 = []
auch12 = []
auch13 = []
auch14 = []
auch15 = []
auch16 = []

for i in tqdm(range(len(z1))):
    match9.append(np.take(window1.body_fredo_magnitude_x[i],z1[i][0]))
    match10.append(np.take(window1.body_fredo_magnitude_y[i],z2[i][0]))
    match11.append(np.take(window1.body_fredo_magnitude_z[i],z3[i][0]))
    match12.append(np.take(window1.body_fredo_magnitude_mag[i],z4[i][0]))
    match13.append(np.take(window1.jerk_body_fredo_magnitude_x[i],z5[i][0]))
    match14.append(np.take(window1.jerk_body_fredo_magnitude_y[i],z6[i][0]))
    match15.append(np.take(window1.jerk_body_fredo_magnitude_z[i],z7[i][0]))
    match16.append(np.take(window1.jerk_body_fredo_magnitude_mag[i],z8[i][0]))
    
    auch9.append(np.take(window1.body_fredo_autcor_magnitude_x[i],z1[i][0]))
    auch10.append(np.take(window1.body_fredo_autcor_magnitude_y[i],z1[i][0]))
    auch11.append(np.take(window1.body_fredo_autcor_magnitude_z[i],z1[i][0]))
    auch12.append(np.take(window1.body_fredo_autcor_magnitude_mag[i],z1[i][0]))
    auch13.append(np.take(window1.jerk_body_fredo_autcor_magnitude_x[i],z1[i][0]))
    auch14.append(np.take(window1.jerk_body_fredo_autcor_magnitude_y[i],z1[i][0]))
    auch15.append(np.take(window1.jerk_body_fredo_autcor_magnitude_z[i],z1[i][0]))
    auch16.append(np.take(window1.jerk_body_fredo_autcor_magnitude_mag[i],z1[i][0]))
    
window1['peak_magnitude_x'] = match9
window1['peak_magnitude_y'] = match10
window1['peak_magnitude_z'] = match11
window1['peak_magnitude_mag'] = match12
window1['jerk_peak_magnitude_x'] = match13
window1['jerk_peak_magnitude_y'] = match14
window1['jerk_peak_magnitude_z'] = match15
window1['jerk_peak_magnitude_mag'] = match16
window1['autcor_peak_magnitude_x'] = auch9
window1['autcor_peak_magnitude_y'] = auch10
window1['autcor_peak_magnitude_z'] = auch11
window1['autcor_peak_magnitude_mag'] = auch12
window1['jerk_autcor_peak_magnitude_x'] = auch13
window1['jerk_autcor_peak_magnitude_y'] = auch14
window1['jerk_autcor_peak_magnitude_z'] = auch15
window1['jerk_autcor_peak_magnitude_mag'] = auch16

#Showing peak frequencies and magnitudes 

ind = detect_peaks(window1['body_fredo_magnitude_x'][0], show=True)
print(ind)



# Central frequency


window1['central_frequency_x'] = window1.peak_frequencies_x.apply(lambda x : np.median(x))
window1['central_frequency_y'] = window1.peak_frequencies_y.apply(lambda x : np.median(x))
window1['central_frequency_z'] = window1.peak_frequencies_z.apply(lambda x : np.median(x))
window1['central_frequency_mag'] = window1.peak_frequencies_mag.apply(lambda x : np.median(x))

window1['jerk_central_frequency_x'] = window1.jerk_peak_frequencies_x.apply(lambda x : np.median(x))
window1['jerk_central_frequency_y'] = window1.jerk_peak_frequencies_y.apply(lambda x : np.median(x))
window1['jerk_central_frequency_z'] = window1.jerk_peak_frequencies_z.apply(lambda x : np.median(x))
window1['jerk_central_frequency_mag'] = window1.jerk_peak_frequencies_mag.apply(lambda x : np.median(x))

window1['autcor_central_frequency_x'] = window1.autcor_peak_frequencies_x.apply(lambda x : np.median(x))
window1['autcor_central_frequency_y'] = window1.autcor_peak_frequencies_y.apply(lambda x : np.median(x))
window1['autcor_central_frequency_z'] = window1.autcor_peak_frequencies_z.apply(lambda x : np.median(x))
window1['autcor_central_frequency_mag'] = window1.autcor_peak_frequencies_mag.apply(lambda x : np.median(x))

window1['jerk_autcor_central_frequency_x'] = window1.jerk_autcor_peak_frequencies_x.apply(lambda x : np.median(x))
window1['jerk_autcor_central_frequency_y'] = window1.jerk_autcor_peak_frequencies_y.apply(lambda x : np.median(x))
window1['jerk_autcor_central_frequency_z'] = window1.jerk_autcor_peak_frequencies_z.apply(lambda x : np.median(x))
window1['jerk_autcor_central_frequency_mag'] = window1.jerk_autcor_peak_frequencies_mag.apply(lambda x : np.median(x))



# central magnitude



window1['central_magnitude_x'] = window1.peak_magnitude_x.apply(lambda x : np.median(x))
window1['central_magnitude_y'] = window1.peak_magnitude_y.apply(lambda x : np.median(x))
window1['central_magnitude_z'] = window1.peak_magnitude_z.apply(lambda x : np.median(x))
window1['central_magnitude_mag'] = window1.peak_magnitude_mag.apply(lambda x : np.median(x))

window1['jerk_central_magnitude_x'] = window1.jerk_peak_magnitude_x.apply(lambda x : np.median(x))
window1['jerk_central_magnitude_y'] = window1.jerk_peak_magnitude_y.apply(lambda x : np.median(x))
window1['jerk_central_magnitude_z'] = window1.jerk_peak_magnitude_z.apply(lambda x : np.median(x))
window1['jerk_central_magnitude_mag'] = window1.jerk_peak_magnitude_mag.apply(lambda x : np.median(x))

window1['autcor_central_magnitude_x'] = window1.autcor_peak_magnitude_x.apply(lambda x : np.median(x))
window1['autcor_central_magnitude_y'] = window1.autcor_peak_magnitude_y.apply(lambda x : np.median(x))
window1['autcor_central_magnitude_z'] = window1.autcor_peak_magnitude_z.apply(lambda x : np.median(x))
window1['autcor_central_magnitude_mag'] = window1.autcor_peak_magnitude_mag.apply(lambda x : np.median(x))

window1['jerk_autcor_central_magnitude_x'] = window1.jerk_autcor_peak_magnitude_x.apply(lambda x : np.median(x))
window1['jerk_autcor_central_magnitude_y'] = window1.jerk_autcor_peak_magnitude_y.apply(lambda x : np.median(x))
window1['jerk_autcor_central_magnitude_z'] = window1.jerk_autcor_peak_magnitude_z.apply(lambda x : np.median(x))
window1['jerk_autcor_central_magnitude_mag'] = window1.jerk_autcor_peak_magnitude_mag.apply(lambda x : np.median(x))


# No of peaks


window1['no_of_peaks_x'] = window1.peak_frequencies_x.apply(lambda x : len(x))
window1['no_of_peaks_y'] = window1.peak_frequencies_y.apply(lambda x : len(x))
window1['no_of_peaks_z'] = window1.peak_frequencies_z.apply(lambda x : len(x))
window1['no_of_peaks_mag'] = window1.peak_frequencies_mag.apply(lambda x : len(x))

window1['jerk_no_of_peaks_x'] = window1.jerk_peak_frequencies_x.apply(lambda x : len(x))
window1['jerk_no_of_peaks_y'] = window1.jerk_peak_frequencies_y.apply(lambda x : len(x))
window1['jerk_no_of_peaks_z'] = window1.jerk_peak_frequencies_z.apply(lambda x : len(x))
window1['jerk_no_of_peaks_mag'] = window1.jerk_peak_frequencies_mag.apply(lambda x : len(x))

window1['autcor_no_of_peaks_x'] = window1.autcor_peak_frequencies_x.apply(lambda x : len(x))
window1['autcor_no_of_peaks_y'] = window1.autcor_peak_frequencies_y.apply(lambda x : len(x))
window1['autcor_no_of_peaks_z'] = window1.autcor_peak_frequencies_z.apply(lambda x : len(x))
window1['autcor_no_of_peaks_mag'] = window1.autcor_peak_frequencies_mag.apply(lambda x : len(x))

window1['jerk_autcor_no_of_peaks_x'] = window1.jerk_autcor_peak_frequencies_x.apply(lambda x : len(x))
window1['jerk_autcor_no_of_peaks_y'] = window1.jerk_autcor_peak_frequencies_y.apply(lambda x : len(x))
window1['jerk_autcor_no_of_peaks_z'] = window1.jerk_autcor_peak_frequencies_z.apply(lambda x : len(x))
window1['jerk_autcor_no_of_peaks_mag'] = window1.jerk_autcor_peak_frequencies_mag.apply(lambda x : len(x))



# Peak frequency


def peak_freq(x):
    if len(x) == 0:
        m = 0
        return m
    else :
        return np.max(x)
        


window1['peak_frequency_x'] = window1.peak_frequencies_x.apply(lambda x : peak_freq(x))
window1['peak_frequency_y'] = window1.peak_frequencies_y.apply(lambda x : peak_freq(x))
window1['peak_frequency_z'] = window1.peak_frequencies_z.apply(lambda x : peak_freq(x))
window1['peak_frequency_mag'] = window1.peak_frequencies_mag.apply(lambda x : peak_freq(x))

window1['jerk_peak_frequency_x'] = window1.jerk_peak_frequencies_x.apply(lambda x : peak_freq(x))
window1['jerk_peak_frequency_y'] = window1.jerk_peak_frequencies_y.apply(lambda x : peak_freq(x))
window1['jerk_peak_frequency_z'] = window1.jerk_peak_frequencies_z.apply(lambda x : peak_freq(x))
window1['jerk_peak_frequency_mag'] = window1.jerk_peak_frequencies_mag.apply(lambda x : peak_freq(x))

window1['autcor_peak_frequency_x'] = window1.autcor_peak_frequencies_x.apply(lambda x : peak_freq(x))
window1['autcor_peak_frequency_y'] = window1.autcor_peak_frequencies_y.apply(lambda x : peak_freq(x))
window1['autcor_peak_frequency_z'] = window1.autcor_peak_frequencies_z.apply(lambda x : peak_freq(x))
window1['autcor_peak_frequency_mag'] = window1.autcor_peak_frequencies_mag.apply(lambda x : peak_freq(x))

window1['jerk_autcor_peak_frequency_x'] = window1.jerk_autcor_peak_frequencies_x.apply(lambda x : peak_freq(x))
window1['jerk_autcor_peak_frequency_y'] = window1.jerk_autcor_peak_frequencies_y.apply(lambda x : peak_freq(x))
window1['jerk_autcor_peak_frequency_z'] = window1.jerk_autcor_peak_frequencies_z.apply(lambda x : peak_freq(x))
window1['jerk_autcor_peak_frequency_mag'] = window1.jerk_autcor_peak_frequencies_mag.apply(lambda x : peak_freq(x))


# peak magnitude


window1['peak_mag_x'] = window1.peak_magnitude_x.apply(lambda x : peak_freq(x))
window1['peak_mag_y'] = window1.peak_magnitude_y.apply(lambda x : peak_freq(x))
window1['peak_mag_z'] = window1.peak_magnitude_z.apply(lambda x : peak_freq(x))
window1['peak_mag_mag'] = window1.peak_magnitude_mag.apply(lambda x : peak_freq(x))

window1['jerk_peak_mag_x'] = window1.jerk_peak_magnitude_x.apply(lambda x : peak_freq(x))
window1['jerk_peak_mag_y'] = window1.jerk_peak_magnitude_y.apply(lambda x : peak_freq(x))
window1['jerk_peak_mag_z'] = window1.jerk_peak_magnitude_z.apply(lambda x : peak_freq(x))
window1['jerk_peak_mag_mag'] = window1.jerk_peak_magnitude_mag.apply(lambda x : peak_freq(x))

window1['autcor_peak_mag_x'] = window1.autcor_peak_magnitude_x.apply(lambda x : peak_freq(x))
window1['autcor_peak_mag_y'] = window1.autcor_peak_magnitude_y.apply(lambda x : peak_freq(x))
window1['autcor_peak_mag_z'] = window1.autcor_peak_magnitude_z.apply(lambda x : peak_freq(x))
window1['autcor_peak_mag_mag'] = window1.autcor_peak_magnitude_mag.apply(lambda x : peak_freq(x))

window1['jerk_autcor_peak_mag_x'] = window1.jerk_autcor_peak_magnitude_x.apply(lambda x : peak_freq(x))
window1['jerk_autcor_peak_mag_y'] = window1.jerk_autcor_peak_magnitude_y.apply(lambda x : peak_freq(x))
window1['jerk_autcor_peak_mag_z'] = window1.jerk_autcor_peak_magnitude_z.apply(lambda x : peak_freq(x))
window1['jerk_autcor_peak_mag_mag'] = window1.jerk_autcor_peak_magnitude_mag.apply(lambda x : peak_freq(x))


# First 5 fft co_efficient 


window1['first_5_fft_co_x1'] = window1.body_fredo_fft_magnitude_x.apply(lambda x : x[0])
window1['first_5_fft_co_x2'] = window1.body_fredo_fft_magnitude_x.apply(lambda x : x[1])
window1['first_5_fft_co_x3'] = window1.body_fredo_fft_magnitude_x.apply(lambda x : x[2])
window1['first_5_fft_co_x4'] = window1.body_fredo_fft_magnitude_x.apply(lambda x : x[3])
window1['first_5_fft_co_x5'] = window1.body_fredo_fft_magnitude_x.apply(lambda x : x[4])

window1['first_5_fft_co_y1'] = window1.body_fredo_fft_magnitude_y.apply(lambda x : x[0])
window1['first_5_fft_co_y2'] = window1.body_fredo_fft_magnitude_y.apply(lambda x : x[1])
window1['first_5_fft_co_y3'] = window1.body_fredo_fft_magnitude_y.apply(lambda x : x[2])
window1['first_5_fft_co_y4'] = window1.body_fredo_fft_magnitude_y.apply(lambda x : x[3])
window1['first_5_fft_co_y5'] = window1.body_fredo_fft_magnitude_y.apply(lambda x : x[4])

window1['first_5_fft_co_z1'] = window1.body_fredo_fft_magnitude_z.apply(lambda x : x[0])
window1['first_5_fft_co_z2'] = window1.body_fredo_fft_magnitude_z.apply(lambda x : x[1])
window1['first_5_fft_co_z3'] = window1.body_fredo_fft_magnitude_z.apply(lambda x : x[2])
window1['first_5_fft_co_z4'] = window1.body_fredo_fft_magnitude_z.apply(lambda x : x[3])
window1['first_5_fft_co_z5'] = window1.body_fredo_fft_magnitude_z.apply(lambda x : x[4])

window1['first_5_fft_co_mag1'] = window1.body_fredo_fft_magnitude_mag.apply(lambda x : x[0])
window1['first_5_fft_co_mag2'] = window1.body_fredo_fft_magnitude_mag.apply(lambda x : x[1])
window1['first_5_fft_co_mag3'] = window1.body_fredo_fft_magnitude_mag.apply(lambda x : x[2])
window1['first_5_fft_co_mag4'] = window1.body_fredo_fft_magnitude_mag.apply(lambda x : x[3])
window1['first_5_fft_co_mag5'] = window1.body_fredo_fft_magnitude_mag.apply(lambda x : x[4])

window1['jerk_first_5_fft_co_x1'] = window1.jerk_body_fredo_fft_magnitude_x.apply(lambda x : x[0])
window1['jerk_first_5_fft_co_x2'] = window1.jerk_body_fredo_fft_magnitude_x.apply(lambda x : x[1])
window1['jerk_first_5_fft_co_x3'] = window1.jerk_body_fredo_fft_magnitude_x.apply(lambda x : x[2])
window1['jerk_first_5_fft_co_x4'] = window1.jerk_body_fredo_fft_magnitude_x.apply(lambda x : x[3])
window1['jerk_first_5_fft_co_x5'] = window1.jerk_body_fredo_fft_magnitude_x.apply(lambda x : x[4])

window1['jerk_first_5_fft_co_y1'] = window1.jerk_body_fredo_fft_magnitude_y.apply(lambda x : x[0])
window1['jerk_first_5_fft_co_y2'] = window1.jerk_body_fredo_fft_magnitude_y.apply(lambda x : x[1])
window1['jerk_first_5_fft_co_y3'] = window1.jerk_body_fredo_fft_magnitude_y.apply(lambda x : x[2])
window1['jerk_first_5_fft_co_y4'] = window1.jerk_body_fredo_fft_magnitude_y.apply(lambda x : x[3])
window1['jerk_first_5_fft_co_y5'] = window1.jerk_body_fredo_fft_magnitude_y.apply(lambda x : x[4])

window1['jerk_first_5_fft_co_z1'] = window1.jerk_body_fredo_fft_magnitude_z.apply(lambda x : x[0])
window1['jerk_first_5_fft_co_z2'] = window1.jerk_body_fredo_fft_magnitude_z.apply(lambda x : x[1])
window1['jerk_first_5_fft_co_z3'] = window1.jerk_body_fredo_fft_magnitude_z.apply(lambda x : x[2])
window1['jerk_first_5_fft_co_z4'] = window1.jerk_body_fredo_fft_magnitude_z.apply(lambda x : x[3])
window1['jerk_first_5_fft_co_z5'] = window1.jerk_body_fredo_fft_magnitude_z.apply(lambda x : x[4])

window1['jerk_first_5_fft_co_mag1'] = window1.jerk_body_fredo_fft_magnitude_mag.apply(lambda x : x[0])
window1['jerk_first_5_fft_co_mag2'] = window1.jerk_body_fredo_fft_magnitude_mag.apply(lambda x : x[1])
window1['jerk_first_5_fft_co_mag3'] = window1.jerk_body_fredo_fft_magnitude_mag.apply(lambda x : x[2])
window1['jerk_first_5_fft_co_mag4'] = window1.jerk_body_fredo_fft_magnitude_mag.apply(lambda x : x[3])
window1['jerk_first_5_fft_co_mag5'] = window1.jerk_body_fredo_fft_magnitude_mag.apply(lambda x : x[4])




# Determining time domain features

In [ ]:
## RMS 


window1['acc_x_rms'] = window1.acc_x.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['acc_y_rms'] = window1.acc_y.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['acc_z_rms'] = window1.acc_z.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['acc_mag_rms'] = window1.magnitude.apply(lambda x : np.sqrt(np.mean(x**2)))

window1['gra_x_rms'] = window1.gra_x.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['gra_y_rms'] = window1.gra_y.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['gra_z_rms'] = window1.gra_z.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['gra_mag_rms'] = window1.gra_mag.apply(lambda x : np.sqrt(np.mean(x**2)))

window1['body_x_rms'] = window1.body_x.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['body_y_rms'] = window1.body_y.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['body_z_rms'] = window1.body_z.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['body_mag_rms'] = window1.body_mag.apply(lambda x : np.sqrt(np.mean(x**2)))

window1['jerk_x_rms'] = window1.jerk_x.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['jerk_y_rms'] = window1.jerk_y.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['jerk_z_rms'] = window1.jerk_z.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['jerk_mag_rms'] = window1.jerk_magnitude.apply(lambda x : np.sqrt(np.mean(x**2)))

window1['jerk_body_x_rms'] = window1.jerk_body_x.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['jerk_body_y_rms'] = window1.jerk_body_y.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['jerk_body_z_rms'] = window1.jerk_body_z.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['jerk_body_mag_rms'] = window1.jerk_body_mag.apply(lambda x : np.sqrt(np.mean(x**2)))

window1['wave_appro_x_rms'] = window1.wave_appro_x.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['wave_appro_y_rms'] = window1.wave_appro_y.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['wave_appro_z_rms'] = window1.wave_appro_z.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['wave_appro_mag_rms'] = window1.wave_appro_mag.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['wave_deltail_x_rms'] = window1.wave_details_x.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['wave_deltail_y_rms'] = window1.wave_details_y.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['wave_deltail_z_rms'] = window1.wave_details_z.apply(lambda x : np.sqrt(np.mean(x**2)))
window1['wave_deltail_mag_rms'] = window1.wave_details_mag.apply(lambda x : np.sqrt(np.mean(x**2)))



#  Min


window1['x_min'] = window1.acc_x.apply(min)
window1['y_min'] = window1.acc_x.apply(min)
window1['z_min'] = window1.acc_x.apply(min)
window1['mag_min'] = window1.magnitude.apply(min)

window1['gra_x_min'] = window1.gra_x.apply(min)
window1['gra_y_min'] = window1.gra_y.apply(min)
window1['gra_z_min'] = window1.gra_z.apply(min)
window1['gra_mag_min'] = window1.gra_mag.apply(min)

window1['body_x_min'] = window1.body_x.apply(min)
window1['body_y_min'] = window1.body_y.apply(min)
window1['body_z_min'] = window1.body_z.apply(min)
window1['body_mag_min'] = window1.body_mag.apply(min)

window1['jerk_x_min'] = window1.jerk_x.apply(min)
window1['jerk_y_min'] = window1.jerk_y.apply(min)
window1['jerk_x_min'] = window1.jerk_z.apply(min)
window1['jerk_magnitude_min'] = window1.jerk_magnitude.apply(min)

window1['jerk_body_x_min'] = window1.jerk_body_x.apply(min)
window1['jerk_body_y_min'] = window1.jerk_body_y.apply(min)
window1['jerk_body_z_min'] = window1.jerk_body_z.apply(min)
window1['jerk_body_mag_min'] = window1.jerk_body_mag.apply(min)


window1['wave_appro_x_min'] = window1.wave_appro_x.apply(min)
window1['wave_appro_y_min'] = window1.wave_appro_y.apply(min)
window1['wave_appro_z_min'] = window1.wave_appro_z.apply(min)
window1['wave_appro_mag_min'] = window1.wave_appro_mag.apply(min)
window1['wave_detail_x_min'] = window1.wave_details_x.apply(min)
window1['wave_detail_y_min'] = window1.wave_details_y.apply(min)
window1['wave_detail_z_min'] = window1.wave_details_z.apply(min)
window1['wave_detail_mag_min'] = window1.wave_details_mag.apply(min)



#window1['jerk_fredo_x_min'] = window1.jerk_fredo_x.apply(min)
#window1['jerk_fredo_y_min'] = window1.jerk_fredo_y.apply(min)
#window1['jerk_fredo_z_min'] = window1.jerk_fredo_z.apply(min)


# Max


window1['x_max'] = window1.acc_x.apply(max)
window1['y_max'] = window1.acc_x.apply(max)
window1['z_max'] = window1.acc_x.apply(max)
window1['mag_max'] = window1.magnitude.apply(max)

window1['gra_x_max'] = window1.gra_x.apply(max)
window1['gra_y_max'] = window1.gra_y.apply(max)
window1['gra_z_max'] = window1.gra_z.apply(max)
window1['gra_mag_max'] = window1.gra_mag.apply(max)

window1['body_x_max'] = window1.body_x.apply(max)
window1['body_y_max'] = window1.body_y.apply(max)
window1['body_z_max'] = window1.body_z.apply(max)
window1['body_mag_max'] = window1.body_mag.apply(max)

window1['jerk_x_max'] = window1.jerk_x.apply(max)
window1['jerk_y_max'] = window1.jerk_y.apply(max)
window1['jerk_z_max'] = window1.jerk_z.apply(max)
window1['jerk_magnitude_max'] = window1.jerk_magnitude.apply(max)

window1['jerk_body_x_max'] = window1.jerk_body_x.apply(max)
window1['jerk_body_y_max'] = window1.jerk_body_y.apply(max)
window1['jerk_body_z_max'] = window1.jerk_body_z.apply(max)
window1['jerk_body_mag_max'] = window1.jerk_body_mag.apply(max)


window1['wave_appro_x_max'] = window1.wave_appro_x.apply(max)
window1['wave_appro_y_max'] = window1.wave_appro_y.apply(max)
window1['wave_appro_z_max'] = window1.wave_appro_z.apply(max)
window1['wave_appro_mag_max'] = window1.wave_appro_mag.apply(max)

window1['wave_detail_x_max'] = window1.wave_details_x.apply(max)
window1['wave_detail_y_max'] = window1.wave_details_y.apply(max)
window1['wave_detail_z_max'] = window1.wave_details_z.apply(max)
window1['wave_detail_mag_max'] = window1.wave_details_mag.apply(max)

#window1['jerk_fredo_x_max'] = window1.jerk_fredo_x.apply(max)
#window1['jerk_fredo_y_max'] = window1.jerk_fredo_y.apply(max)
#window1['jerk_fredo_z_max'] = window1.jerk_fredo_z.apply(max)

# Mean


window1['x_mean'] = window1.acc_x.apply(lambda x : sum(x)/len(x))
window1['y_mean'] = window1.acc_y.apply(lambda x : sum(x)/len(x))
window1['z_mean'] = window1.acc_z.apply(lambda x : sum(x)/len(x))
window1['mag_mean'] = window1.magnitude.apply(lambda x : sum(x)/len(x))

window1['gra_x_mean'] = window1.gra_x.apply(lambda x : sum(x)/len(x))
window1['gra_y_mean'] = window1.gra_y.apply(lambda x : sum(x)/len(x))
window1['gra_z_mean'] = window1.gra_z.apply(lambda x : sum(x)/len(x))
window1['gra_mag_mean'] = window1.gra_mag.apply(lambda x : sum(x)/len(x))

window1['body_x_mean'] = window1.body_x.apply(lambda x : sum(x)/len(x))
window1['body_y_mean'] = window1.body_y.apply(lambda x : sum(x)/len(x))
window1['body_z_mean'] = window1.body_z.apply(lambda x : sum(x)/len(x))
window1['body_mag_mean'] = window1.body_mag.apply(lambda x : sum(x)/len(x))

window1['jerk_x_mean'] = window1.jerk_x.apply(lambda x : sum(x)/len(x))
window1['jerk_y_mean'] = window1.jerk_y.apply(lambda x : sum(x)/len(x))
window1['jerk_z_mean'] = window1.jerk_z.apply(lambda x : sum(x)/len(x))
window1['jerk_magnitude_mean'] = window1.jerk_magnitude.apply(lambda x : sum(x)/len(x))

window1['jerk_body_x_mean'] = window1.jerk_body_x.apply(lambda x : sum(x)/len(x))
window1['jerk_body_y_mean'] = window1.jerk_body_y.apply(lambda x : sum(x)/len(x))
window1['jerk_body_z_mean'] = window1.jerk_body_z.apply(lambda x : sum(x)/len(x))
window1['jerk_body_mag_mean'] = window1.jerk_body_mag.apply(lambda x : sum(x)/len(x))

window1['wave_appro_x_mean'] = window1.wave_appro_x.apply(lambda x : sum(x)/len(x))
window1['wave_appro_y_mean'] = window1.wave_appro_y.apply(lambda x : sum(x)/len(x))
window1['wave_appro_z_mean'] = window1.wave_appro_z.apply(lambda x : sum(x)/len(x))
window1['wave_appro_mag_mean'] = window1.wave_appro_mag.apply(lambda x : sum(x)/len(x))

window1['wave_detail_x_mean'] = window1.wave_details_x.apply(lambda x : sum(x)/len(x))
window1['wave_detail_y_mean'] = window1.wave_details_y.apply(lambda x : sum(x)/len(x))
window1['wave_detail_z_mean'] = window1.wave_details_z.apply(lambda x : sum(x)/len(x))
window1['wave_detail_mag_mean'] = window1.wave_details_mag.apply(lambda x : sum(x)/len(x))



# Standard Deviation



window1['x_std'] = window1.acc_x.apply(lambda x : np.std(x))
window1['y_std'] = window1.acc_y.apply(lambda x : np.std(x))
window1['z_std'] = window1.acc_z.apply(lambda x : np.std(x))
window1['magnitude_std'] = window1.magnitude.apply(lambda x : np.std(x))

window1['gra_x_std'] = window1.gra_x.apply(lambda x : np.std(x))
window1['gra_y_std'] = window1.gra_y.apply(lambda x : np.std(x))
window1['gra_z_std'] = window1.gra_z.apply(lambda x : np.std(x))
window1['gra_mag_std'] = window1.gra_mag.apply(lambda x : np.std(x))

window1['body_x_std'] = window1.body_x.apply(lambda x : np.std(x))
window1['body_y_std'] = window1.body_y.apply(lambda x : np.std(x))
window1['body_z_std'] = window1.body_z.apply(lambda x : np.std(x))
window1['body_mag_std'] = window1.body_mag.apply(lambda x : np.std(x))

window1['jerk_x_std'] = window1.jerk_x.apply(lambda x : np.std(x))
window1['jerk_y_std'] = window1.jerk_y.apply(lambda x : np.std(x))
window1['jerk_z_std'] = window1.jerk_z.apply(lambda x : np.std(x))
window1['jerk_magnitude_std'] = window1.jerk_magnitude.apply(lambda x : np.std(x))

window1['jerk_body_x_std'] = window1.jerk_body_x.apply(lambda x : np.std(x))
window1['jerk_body_y_std'] = window1.jerk_body_y.apply(lambda x : np.std(x))
window1['jerk_body_z_std'] = window1.jerk_body_z.apply(lambda x : np.std(x))
window1['jerk_body_mag_std'] = window1.jerk_body_mag.apply(lambda x : np.std(x))

window1['wave_appro_x_std'] = window1.wave_appro_x.apply(lambda x : np.std(x))
window1['wave_appro_y_std'] = window1.wave_appro_y.apply(lambda x : np.std(x))
window1['wave_appro_z_std'] = window1.wave_appro_z.apply(lambda x : np.std(x))
window1['wave_appro_mag_std'] = window1.wave_appro_mag.apply(lambda x : np.std(x))
window1['wave_detail_x_std'] = window1.wave_details_x.apply(lambda x : np.std(x))
window1['wave_detail_y_std'] = window1.wave_details_y.apply(lambda x : np.std(x))
window1['wave_detail_z_std'] = window1.wave_details_z.apply(lambda x : np.std(x))
window1['wave_detail_mag_std'] = window1.wave_details_mag.apply(lambda x : np.std(x))


# Variance


window1['x_var'] = window1.acc_x.apply(lambda x : np.var(x))
window1['y_var'] = window1.acc_y.apply(lambda x : np.var(x))
window1['z_var'] = window1.acc_z.apply(lambda x : np.var(x))
window1['magnitude_var'] = window1.magnitude.apply(lambda x : np.var(x))

window1['gra_x_var'] = window1.gra_x.apply(lambda x : np.var(x))
window1['gra_y_var'] = window1.gra_y.apply(lambda x : np.var(x))
window1['gra_z_var'] = window1.gra_z.apply(lambda x : np.var(x))
window1['gra_mag_var'] = window1.gra_mag.apply(lambda x : np.var(x))

window1['body_x_var'] = window1.body_x.apply(lambda x : np.var(x))
window1['body_y_var'] = window1.body_y.apply(lambda x : np.var(x))
window1['body_z_var'] = window1.body_z.apply(lambda x : np.var(x))
window1['body_mag_var'] = window1.body_mag.apply(lambda x : np.var(x))

window1['jerk_x_var'] = window1.jerk_x.apply(lambda x : np.var(x))
window1['jerk_y_var'] = window1.jerk_y.apply(lambda x : np.var(x))
window1['jerk_z_var'] = window1.jerk_z.apply(lambda x : np.var(x))
window1['jerk_magnitude_var'] = window1.jerk_magnitude.apply(lambda x : np.var(x))

window1['jerk_body_x_var'] = window1.jerk_body_x.apply(lambda x : np.var(x))
window1['jerk_body_y_var'] = window1.jerk_body_y.apply(lambda x : np.var(x))
window1['jerk_body_z_var'] = window1.jerk_body_z.apply(lambda x : np.var(x))
window1['jerk_body_mag_var'] = window1.jerk_body_mag.apply(lambda x : np.var(x))

window1['wave_appro_x_var'] = window1.wave_appro_x.apply(lambda x : np.var(x))
window1['wave_appro_y_var'] = window1.wave_appro_y.apply(lambda x : np.var(x))
window1['wave_appro_z_var'] = window1.wave_appro_z.apply(lambda x : np.var(x))
window1['wave_appro_mag_var'] = window1.wave_appro_mag.apply(lambda x : np.var(x))

window1['wave_detail_x_var'] = window1.wave_details_x.apply(lambda x : np.var(x))
window1['wave_detail_y_var'] = window1.wave_details_y.apply(lambda x : np.var(x))
window1['wave_detail_z_var'] = window1.wave_details_z.apply(lambda x : np.var(x))
window1['wave_detail_mag_var'] = window1.wave_details_mag.apply(lambda x : np.var(x))





# Median



window1['x_median'] = window1.acc_x.apply(lambda x : np.median(x))
window1['y_median'] = window1.acc_y.apply(lambda x : np.median(x))
window1['z_median'] = window1.acc_z.apply(lambda x : np.median(x))
window1['magnitude_median'] = window1.magnitude.apply(lambda x : np.median(x))

window1['gra_x_median'] = window1.gra_x.apply(lambda x : np.median(x))
window1['gra_y_median'] = window1.gra_y.apply(lambda x : np.median(x))
window1['gra_z_median'] = window1.gra_z.apply(lambda x : np.median(x))
window1['gra_mag_median'] = window1.gra_mag.apply(lambda x : np.median(x))

window1['body_x_median'] = window1.body_x.apply(lambda x : np.median(x))
window1['body_y_median'] = window1.body_y.apply(lambda x : np.median(x))
window1['body_z_median'] = window1.body_z.apply(lambda x : np.median(x))
window1['body_mag_median'] = window1.body_mag.apply(lambda x : np.median(x))

window1['jerk_x_median'] = window1.jerk_x.apply(lambda x : np.median(x))
window1['jerk_y_median'] = window1.jerk_y.apply(lambda x : np.median(x))
window1['jerk_z_median'] = window1.jerk_z.apply(lambda x : np.median(x))
window1['jerk_magnitude_median'] = window1.jerk_magnitude.apply(lambda x : np.median(x))

window1['jerk_body_x_median'] = window1.jerk_body_x.apply(lambda x : np.median(x))
window1['jerk_body_y_median'] = window1.jerk_body_y.apply(lambda x : np.median(x))
window1['jerk_body_z_median'] = window1.jerk_body_z.apply(lambda x : np.median(x))
window1['jerk_body_mag_median'] = window1.jerk_body_mag.apply(lambda x : np.median(x))

window1['wave_appro_x_median'] = window1.wave_appro_x.apply(lambda x : np.median(x))
window1['wave_appro_y_median'] = window1.wave_appro_y.apply(lambda x : np.median(x))
window1['wave_appro_z_median'] = window1.wave_appro_z.apply(lambda x : np.median(x))
window1['wave_appro_mag_median'] = window1.wave_appro_mag.apply(lambda x : np.median(x))
window1['wave_detail_x_median'] = window1.wave_details_x.apply(lambda x : np.median(x))
window1['wave_detail_y_median'] = window1.wave_details_y.apply(lambda x : np.median(x))
window1['wave_detail_z_median'] = window1.wave_details_z.apply(lambda x : np.median(x))
window1['wave_detail_mag_median'] = window1.wave_details_mag.apply(lambda x : np.median(x))




# Interquartile Range



window1['third_quartile_x'] = window1.acc_x.apply(lambda x : np.percentile(x,75))
window1['first_quartile_x'] = window1.acc_x.apply(lambda x : np.percentile(x,25))
window1['x_IQR'] = window1['third_quartile_x'] - window1['first_quartile_x']

window1['third_quartile_y'] = window1.acc_y.apply(lambda x : np.percentile(x,75))
window1['first_quartile_y'] = window1.acc_y.apply(lambda x : np.percentile(x,25))
window1['y_IQR'] = window1['third_quartile_y'] - window1['first_quartile_y']

window1['third_quartile_z'] = window1.acc_z.apply(lambda x : np.percentile(x,75))
window1['first_quartile_z'] = window1.acc_z.apply(lambda x : np.percentile(x,25))
window1['z_IQR'] = window1['third_quartile_z'] - window1['first_quartile_z']

window1['third_quartile_magnitude'] = window1.magnitude.apply(lambda x : np.percentile(x,75))
window1['first_quartile_magnitude'] = window1.magnitude.apply(lambda x : np.percentile(x,25))
window1['magnitude_IQR'] = window1['third_quartile_y'] - window1['first_quartile_y']

#

window1['third_quartile_gra_x'] = window1.gra_x.apply(lambda x : np.percentile(x,75))
window1['first_quartile_gra_x'] = window1.gra_x.apply(lambda x : np.percentile(x,25))
window1['gra_x_IQR'] = window1['third_quartile_gra_x'] - window1['first_quartile_gra_x']

window1['third_quartile_gra_y'] = window1.gra_y.apply(lambda x : np.percentile(x,75))
window1['first_quartile_gra_y'] = window1.gra_y.apply(lambda x : np.percentile(x,25))
window1['gra_y_IQR'] = window1['third_quartile_gra_y'] - window1['first_quartile_gra_y']

window1['third_quartile_gra_z'] = window1.gra_z.apply(lambda x : np.percentile(x,75))
window1['first_quartile_gra_z'] = window1.gra_z.apply(lambda x : np.percentile(x,25))
window1['gra_z_IQR'] = window1['third_quartile_gra_z'] - window1['first_quartile_gra_z']

window1['third_quartile_gra_mag'] = window1.gra_mag.apply(lambda x : np.percentile(x,75))
window1['first_quartile_gra_mag'] = window1.gra_mag.apply(lambda x : np.percentile(x,25))
window1['gra_mag_IQR'] = window1['third_quartile_gra_mag'] - window1['first_quartile_gra_mag']

#

window1['third_quartile_body_x'] = window1.body_x.apply(lambda x : np.percentile(x,75))
window1['first_quartile_body_x'] = window1.body_x.apply(lambda x : np.percentile(x,25))
window1['body_x_IQR'] = window1['third_quartile_body_x'] - window1['first_quartile_body_x']

window1['third_quartile_body_y'] = window1.body_y.apply(lambda x : np.percentile(x,75))
window1['first_quartile_body_y'] = window1.body_y.apply(lambda x : np.percentile(x,25))
window1['body_y_IQR'] = window1['third_quartile_body_y'] - window1['first_quartile_body_y']

window1['third_quartile_body_z'] = window1.body_z.apply(lambda x : np.percentile(x,75))
window1['first_quartile_body_z'] = window1.body_z.apply(lambda x : np.percentile(x,25))
window1['body_z_IQR'] = window1['third_quartile_body_z'] - window1['first_quartile_body_z']

window1['third_quartile_body_mag'] = window1.body_mag.apply(lambda x : np.percentile(x,75))
window1['first_quartile_body_mag'] = window1.body_mag.apply(lambda x : np.percentile(x,25))
window1['body_mag_IQR'] = window1['third_quartile_body_mag'] - window1['first_quartile_body_mag']

#

window1['third_quartile_jerk_x'] = window1.jerk_x.apply(lambda x : np.percentile(x,75))
window1['first_quartile_jerk_x'] = window1.jerk_x.apply(lambda x : np.percentile(x,25))
window1['jerk_x_IQR'] = window1['third_quartile_jerk_x'] - window1['first_quartile_jerk_x']

window1['third_quartile_jerk_y'] = window1.jerk_y.apply(lambda x : np.percentile(x,75))
window1['first_quartile_jerk_y'] = window1.jerk_y.apply(lambda x : np.percentile(x,25))
window1['jerk_y_IQR'] = window1['third_quartile_jerk_y'] - window1['first_quartile_jerk_y']

window1['third_quartile_jerk_z'] = window1.jerk_z.apply(lambda x : np.percentile(x,75))
window1['first_quartile_jerk_z'] = window1.jerk_z.apply(lambda x : np.percentile(x,25))
window1['jerk_z_IQR'] = window1['third_quartile_jerk_z'] - window1['first_quartile_jerk_z']

window1['third_quartile_jerk_magnitude'] = window1.jerk_magnitude.apply(lambda x : np.percentile(x,75))
window1['first_quartile_jerk_magnitude'] = window1.jerk_magnitude.apply(lambda x : np.percentile(x,25))
window1['jerk_magnitude_IQR'] = window1['third_quartile_jerk_magnitude'] - window1['first_quartile_jerk_magnitude']

#

window1['third_quartile_jerk_body_x'] = window1.jerk_body_x.apply(lambda x : np.percentile(x,75))
window1['first_quartile_jerk_body_x'] = window1.jerk_body_x.apply(lambda x : np.percentile(x,25))
window1['jerk_body_x_IQR'] = window1['third_quartile_jerk_body_x'] - window1['first_quartile_jerk_body_x']

window1['third_quartile_jerk_body_y'] = window1.jerk_body_y.apply(lambda x : np.percentile(x,75))
window1['first_quartile_jerk_body_y'] = window1.jerk_body_y.apply(lambda x : np.percentile(x,25))
window1['jerk_body_y_IQR'] = window1['third_quartile_jerk_body_y'] - window1['first_quartile_jerk_body_y']

window1['third_quartile_jerk_body_z'] = window1.jerk_body_z.apply(lambda x : np.percentile(x,75))
window1['first_quartile_jerk_body_z'] = window1.jerk_body_z.apply(lambda x : np.percentile(x,25))
window1['jerk_body_z_IQR'] = window1['third_quartile_jerk_body_z'] - window1['first_quartile_jerk_body_z']

window1['third_quartile_jerk_body_mag'] = window1.jerk_body_mag.apply(lambda x : np.percentile(x,75))
window1['first_quartile_jerk_body_mag'] = window1.jerk_body_mag.apply(lambda x : np.percentile(x,25))
window1['jerk_body_mag_IQR'] = window1['third_quartile_jerk_body_mag'] - window1['first_quartile_jerk_body_mag']


window1['wave_appro_third_quartile_x'] = window1.wave_appro_x.apply(lambda x : np.percentile(x,75))
window1['wave_appro_first_quartile_x'] = window1.wave_appro_x.apply(lambda x : np.percentile(x,25))
window1['wave_appro_x_IQR'] = window1['wave_appro_third_quartile_x'] - window1['wave_appro_first_quartile_x']

window1['wave_appro_third_quartile_y'] = window1.wave_appro_y.apply(lambda x : np.percentile(x,75))
window1['wave_appro_first_quartile_y'] = window1.wave_appro_y.apply(lambda x : np.percentile(x,25))
window1['wave_appro_y_IQR'] = window1['wave_appro_third_quartile_y'] - window1['wave_appro_first_quartile_y']

window1['wave_appro_third_quartile_z'] = window1.wave_appro_z.apply(lambda x : np.percentile(x,75))
window1['wave_appro_first_quartile_z'] = window1.wave_appro_z.apply(lambda x : np.percentile(x,25))
window1['wave_appro_z_IQR'] = window1['wave_appro_third_quartile_z'] - window1['wave_appro_first_quartile_z']

window1['wave_appro_third_quartile_mag'] = window1.wave_appro_mag.apply(lambda x : np.percentile(x,75))
window1['wave_appro_first_quartile_mag'] = window1.wave_appro_mag.apply(lambda x : np.percentile(x,25))
window1['wave_appro_mag_IQR'] = window1['wave_appro_third_quartile_mag'] - window1['wave_appro_first_quartile_mag']

window1['wave_detail_third_quartile_x'] = window1.wave_details_x.apply(lambda x : np.percentile(x,75))
window1['wave_detail_first_quartile_x'] = window1.wave_details_x.apply(lambda x : np.percentile(x,25))
window1['wave_detail_x_IQR'] = window1['wave_detail_third_quartile_x'] - window1['wave_detail_first_quartile_x']

window1['wave_detail_third_quartile_y'] = window1.wave_details_y.apply(lambda x : np.percentile(x,75))
window1['wave_detail_first_quartile_y'] = window1.wave_details_y.apply(lambda x : np.percentile(x,25))
window1['wave_detail_y_IQR'] = window1['wave_detail_third_quartile_y'] - window1['wave_detail_first_quartile_y']

window1['wave_detail_third_quartile_z'] = window1.wave_details_z.apply(lambda x : np.percentile(x,75))
window1['wave_detail_first_quartile_z'] = window1.wave_details_z.apply(lambda x : np.percentile(x,25))
window1['wave_detail_z_IQR'] = window1['wave_detail_third_quartile_z'] - window1['wave_detail_first_quartile_z']

window1['wave_detail_third_quartile_mag'] = window1.wave_details_mag.apply(lambda x : np.percentile(x,75))
window1['wave_detail_first_quartile_mag'] = window1.wave_details_mag.apply(lambda x : np.percentile(x,25))
window1['wave_detail_mag_IQR'] = window1['wave_detail_third_quartile_mag'] - window1['wave_detail_first_quartile_mag']





# Peek to peek



window1['x_peek'] = window1.acc_x.apply(lambda x : max(x)-min(x))
window1['y_peek'] = window1.acc_y.apply(lambda x : max(x)-min(x))
window1['z_peek'] = window1.acc_z.apply(lambda x : max(x)-min(x))
window1['magnitude_peek'] = window1.magnitude.apply(lambda x : max(x)-min(x))

window1['gra_x_peek'] = window1.gra_x.apply(lambda x : max(x)-min(x))
window1['gra_y_peek'] = window1.gra_y.apply(lambda x : max(x)-min(x))
window1['gra_z_peek'] = window1.gra_z.apply(lambda x : max(x)-min(x))
window1['gra_mag_peek'] = window1.gra_mag.apply(lambda x : max(x)-min(x))



window1['body_x_peek'] = window1.body_x.apply(lambda x : max(x)-min(x))
window1['body_y_peek'] = window1.body_y.apply(lambda x : max(x)-min(x))
window1['body_z_peek'] = window1.body_z.apply(lambda x : max(x)-min(x))
window1['body_mag_peek'] = window1.body_mag.apply(lambda x : max(x)-min(x))

window1['jerk_x_peek'] = window1.jerk_x.apply(lambda x : max(x)-min(x))
window1['jerk_y_peek'] = window1.jerk_y.apply(lambda x : max(x)-min(x))
window1['jerk_z_peek'] = window1.jerk_z.apply(lambda x : max(x)-min(x))
window1['jerk_magnitude_peek'] = window1.jerk_magnitude.apply(lambda x : max(x)-min(x))

window1['jerk_body_x_peek'] = window1.jerk_body_x.apply(lambda x : max(x)-min(x))
window1['jerk_body_y_peek'] = window1.jerk_body_y.apply(lambda x : max(x)-min(x))
window1['jerk_body_z_peek'] = window1.jerk_body_z.apply(lambda x : max(x)-min(x))
window1['jerk_body_mag_peek'] = window1.jerk_body_mag.apply(lambda x : max(x)-min(x))



# MAD ( Mean Absolute Deviation)


window1['x_mad'] = window1.acc_x.apply(lambda x : np.mean(np.absolute(x-np.mean(x))))
window1['y_mad'] = window1.acc_y.apply(lambda x : np.mean(np.absolute(x-np.mean(x))))
window1['z_mad'] = window1.acc_z.apply(lambda x : np.mean(np.absolute(x-np.mean(x))))
window1['magnitude_mad'] = window1.magnitude.apply(lambda x : np.mean(np.absolute(x-np.mean(x))))


window1['gra_x_mad'] = window1.gra_x.apply(lambda x : np.mean(np.absolute(x-np.mean(x))))
window1['gra_y_mad'] = window1.gra_y.apply(lambda x : np.mean(np.absolute(x-np.mean(x))))
window1['gra_z_mad'] = window1.gra_z.apply(lambda x : np.mean(np.absolute(x-np.mean(x))))
window1['gra_mag_mad'] = window1.gra_mag.apply(lambda x : np.mean(np.absolute(x-np.mean(x))))

window1['body_x_mad'] = window1.body_x.apply(lambda x : np.mean(np.absolute(x-np.mean(x))))
window1['body_y_mad'] = window1.body_y.apply(lambda x : np.mean(np.absolute(x-np.mean(x))))
window1['body_z_mad'] = window1.body_z.apply(lambda x : np.mean(np.absolute(x-np.mean(x))))
window1['body_mag_mad'] = window1.body_mag.apply(lambda x : np.mean(np.absolute(x-np.mean(x))))

window1['jerk_x_mad'] = window1.jerk_x.apply(lambda x : np.mean(np.absolute(x-np.mean(x))))
window1['jerk_y_mad'] = window1.jerk_y.apply(lambda x : np.mean(np.absolute(x-np.mean(x))))
window1['jerk_z_mad'] = window1.jerk_z.apply(lambda x : np.mean(np.absolute(x-np.mean(x))))
window1['jerk_magnitude_mad'] = window1.jerk_magnitude.apply(lambda x : np.mean(np.absolute(x-np.mean(x))))

window1['jerk_body_x_mad'] = window1.jerk_body_x.apply(lambda x : np.mean(np.absolute(x-np.mean(x))))
window1['jerk_body_y_mad'] = window1.jerk_body_y.apply(lambda x : np.mean(np.absolute(x-np.mean(x))))
window1['jerk_body_z_mad'] = window1.jerk_body_z.apply(lambda x : np.mean(np.absolute(x-np.mean(x))))
window1['jerk_body_mag_mad'] = window1.jerk_body_mag.apply(lambda x : np.mean(np.absolute(x-np.mean(x))))


## Velocity

def get_velocity(x, axis = 0, nperseg = 100):
    return np.trapz(x, axis = axis)/nperseg

window1['x_velocity'] = window1.acc_x.apply(lambda x : get_velocity(x,axis=0,nperseg=100))
window1['y_velocity'] = window1.acc_y.apply(lambda x : get_velocity(x,axis=0,nperseg=100))
window1['z_velocity'] = window1.acc_z.apply(lambda x : get_velocity(x,axis=0,nperseg=100))
window1['magnitude_velocity'] = window1.magnitude.apply(lambda x : get_velocity(x,axis=0,nperseg=100))

window1['gra_x_velocity'] = window1.gra_x.apply(lambda x : get_velocity(x,axis=0,nperseg=100))
window1['gra_y_velocity'] = window1.gra_y.apply(lambda x : get_velocity(x,axis=0,nperseg=100))
window1['gra_z_velocity'] = window1.gra_z.apply(lambda x : get_velocity(x,axis=0,nperseg=100))
window1['gra_mag_velocity'] = window1.gra_mag.apply(lambda x : get_velocity(x,axis=0,nperseg=100))

window1['body_x_velocity'] = window1.body_x.apply(lambda x : get_velocity(x,axis=0,nperseg=100))
window1['body_y_velocity'] = window1.body_y.apply(lambda x : get_velocity(x,axis=0,nperseg=100))
window1['body_z_velocity'] = window1.body_z.apply(lambda x : get_velocity(x,axis=0,nperseg=100))
window1['body_mag_velocity'] = window1.body_mag.apply(lambda x : get_velocity(x,axis=0,nperseg=100))


In [ ]:
window1

In [ ]:
## mutual co-relation



def mutual_corr(a,b):    
    corr = []
    for i in tqdm(range(len(a))):
        v = np.corrcoef(a[i],b[i])
        corr.append(v[0,1])
    return corr

window1['acc_x_y_rela'] = mutual_corr(window1.acc_x,window1.acc_y)
window1['acc_x_z_rela'] = mutual_corr(window1.acc_x,window1.acc_z)
window1['acc_y_z_rela'] = mutual_corr(window1.acc_y,window1.acc_z)

window1['jerk_x_y_rela'] = mutual_corr(window1.jerk_x,window1.jerk_y)
window1['jerk_x_z_rela'] = mutual_corr(window1.jerk_x,window1.jerk_z)
window1['jerk_y_z_rela'] = mutual_corr(window1.jerk_y,window1.jerk_z)

window1['gra_x_y_rela'] = mutual_corr(window1.gra_x,window1.gra_y)
window1['gra_x_z_rela'] = mutual_corr(window1.gra_x,window1.gra_z)
window1['gra_y_z_rela'] = mutual_corr(window1.gra_y,window1.gra_z)

window1['body_x_y_rela'] = mutual_corr(window1.body_x,window1.body_y)
window1['body_x_z_rela'] = mutual_corr(window1.body_x,window1.body_z)
window1['body_y_z_rela'] = mutual_corr(window1.body_y,window1.body_z)

window1['jerk_body_x_y_rela'] = mutual_corr(window1.jerk_body_x,window1.jerk_body_y)
window1['jerk_body_x_z_rela'] = mutual_corr(window1.jerk_body_x,window1.jerk_body_z)
window1['jerk_body_y_z_rela'] = mutual_corr(window1.jerk_body_y,window1.jerk_body_z)


## mutual co-varriance


def mutual_covar(a,b):    
    covar = []
    for i in tqdm(range(len(a))):
        v = np.cov(a[i], b[i], bias=True)
        covar.append(v[0,1])
    return covar

window1['acc_x_y_covar'] = mutual_covar(window1.acc_x,window1.acc_y)
window1['acc_x_z_covar'] = mutual_covar(window1.acc_x,window1.acc_z)
window1['acc_y_z_covar'] = mutual_covar(window1.acc_y,window1.acc_z)

window1['jerk_x_y_covar'] = mutual_covar(window1.jerk_x,window1.jerk_y)
window1['jerk_x_z_covar'] = mutual_covar(window1.jerk_x,window1.jerk_z)
window1['jerk_y_z_covar'] = mutual_covar(window1.jerk_y,window1.jerk_z)

window1['gra_x_y_covar'] = mutual_covar(window1.gra_x,window1.gra_y)
window1['gra_x_z_covar'] = mutual_covar(window1.gra_x,window1.gra_z)
window1['gra_y_z_covar'] = mutual_covar(window1.gra_y,window1.gra_z)

window1['body_x_y_covar'] = mutual_covar(window1.body_x,window1.body_y)
window1['body_x_z_covar'] = mutual_covar(window1.body_x,window1.body_z)
window1['body_y_z_covar'] = mutual_covar(window1.body_y,window1.body_z)

window1['jerk_body_x_y_covar'] = mutual_covar(window1.jerk_body_x,window1.jerk_body_y)
window1['jerk_body_x_z_covar'] = mutual_covar(window1.jerk_body_x,window1.jerk_body_z)
window1['jerk_body_y_z_covar'] = mutual_covar(window1.jerk_body_y,window1.jerk_body_z)




In [ ]:
window1

# Dropping the signals from the feature data

In [ ]:
window_fi = window1.drop(['acc_x','acc_y','acc_z','magnitude','gra_x','gra_y','gra_z','gra_mag','body_x','body_y','body_z','body_mag','body_fredo_magnitude_x','body_fredo_magnitude_y','body_fredo_magnitude_z','body_fredo_magnitude_mag','body_fredo_frequency_x','body_fredo_frequency_y','body_fredo_frequency_z','body_fredo_frequency_mag','jerk_x','jerk_y','jerk_z','jerk_magnitude','jerk_body_x','jerk_body_y','jerk_body_z','jerk_body_mag','jerk_body_fredo_magnitude_x','jerk_body_fredo_magnitude_y','jerk_body_fredo_magnitude_z','jerk_body_fredo_magnitude_mag'],axis=1)
window_fi = window_fi.drop(['body_fredo_autcor_magnitude_x','body_fredo_autcor_magnitude_y','body_fredo_autcor_magnitude_z','body_fredo_autcor_frequency_x','body_fredo_autcor_frequency_y','body_fredo_autcor_frequency_z','body_fredo_autcor_magnitude_mag','body_fredo_autcor_frequency_mag','body_fredo_fft_magnitude_x','body_fredo_fft_magnitude_y','body_fredo_fft_magnitude_z','body_fredo_fft_magnitude_mag','jerk_body_fredo_fft_magnitude_x','jerk_body_fredo_fft_magnitude_y','jerk_body_fredo_fft_magnitude_z','jerk_body_fredo_fft_magnitude_mag'],axis=1)
window_fi = window_fi.drop(['jerk_body_fredo_autcor_magnitude_x','jerk_body_fredo_autcor_magnitude_y','jerk_body_fredo_autcor_magnitude_z','jerk_body_fredo_autcor_magnitude_mag','jerk_body_fredo_autcor_frequency_x','jerk_body_fredo_autcor_frequency_y','jerk_body_fredo_autcor_frequency_z','jerk_body_fredo_autcor_frequency_mag','peak_frequencies_x','peak_frequencies_y','peak_frequencies_z','peak_frequencies_mag'],axis=1)

In [ ]:
window_fi = window_fi.drop(['jerk_peak_frequencies_x','jerk_peak_frequencies_y','jerk_peak_frequencies_z','jerk_peak_frequencies_mag','autcor_peak_frequencies_x','autcor_peak_frequencies_y','autcor_peak_frequencies_z','autcor_peak_frequencies_mag','jerk_autcor_peak_frequencies_x','jerk_autcor_peak_frequencies_y','jerk_autcor_peak_frequencies_z','jerk_autcor_peak_frequencies_mag','peak_magnitude_x','peak_magnitude_y','peak_magnitude_z','peak_magnitude_mag','jerk_peak_magnitude_x','jerk_peak_magnitude_y','jerk_peak_magnitude_z','jerk_peak_magnitude_mag'],axis =1)
window_fi = window_fi.drop(['autcor_peak_magnitude_x','autcor_peak_magnitude_y','autcor_peak_magnitude_z','autcor_peak_magnitude_mag','jerk_autcor_peak_magnitude_x','jerk_autcor_peak_magnitude_y','jerk_autcor_peak_magnitude_z','jerk_autcor_peak_magnitude_mag','jerk_body_fredo_frequency_x','jerk_body_fredo_frequency_y','jerk_body_fredo_frequency_z','jerk_body_fredo_frequency_mag','wave_appro_x','wave_appro_y','wave_appro_z','wave_appro_mag','wave_details_x','wave_details_y','wave_details_z','wave_details_mag'],axis =1)

In [ ]:
window_fi

# Checking is there any null or infinite value in the feature DF

In [ ]:
window_fi.isna().sum().sum()

In [ ]:
window_fi.dropna(inplace=True)
window_fi.isna().sum().sum()

In [ ]:
window_fi.isin([np.inf, -np.inf]).sum().sum()

In [ ]:
pickel_out2 = open('feature_user2.pickle','wb')
pickle.dump(window_fi,pickel_out2)
pickel_out2.close()